## Setup

In [ ]:
import pandas as pd
import logging 
import sys
import os
sys.path.insert(0,'./src')


from src.utils.logger import sys_logger

os.makedirs("./final_data", exist_ok=True)

## Landing Zone to Staging

In [ ]:
from src.utils.splitter import split_files

split_files()

## Staging to Tables

In [ ]:
from src.utils.ingestion import run_ingestion

run_ingestion()

## Report Generation

In [ ]:
from matplotlib import pyplot as plt

### Customer Growth

In [ ]:
customers = pd.read_parquet('./final_data/customer.parquet')

dates = customers['effective_from'].unique()

data = []
for date in sorted(dates):
    snapshot = customers[(customers['effective_from'] <= date) & (customers['expiry_date'] > date)]
    data.append({'date': date, 'count': len(snapshot)})

df = pd.DataFrame(data)
df.plot(x='date', y='count')
plt.title('Customer Growth')
plt.xlabel('Date')
plt.ylabel('Number of Customers')
plt.show()

### Sales By Membership

In [ ]:
# Sales By Members
customers = pd.read_parquet('./final_data/customer.parquet')

dates = customers['effective_from'].unique()
sales = pd.read_parquet('./final_data/sales.parquet')
merged = sales.merge(customers, on='customer_id', how='outer')
merged = merged[(merged['effective_from_y'] <= merged['sale_date']) & (merged['expiry_date_y'] > merged['sale_date'])]
data = merged.groupby(['sale_date', 'membership_status'])['total_price'].sum().unstack().fillna(0)

data.plot()
plt.title('Sales by Membership Status')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.show()

### Product Sale Breakdown

In [ ]:
# pivot of most popular products over time from members vs non-members

products = pd.read_parquet('./final_data/product.parquet')
customers = pd.read_parquet('./final_data/customer.parquet')
sales = pd.read_parquet('./final_data/sales.parquet')

merged = sales.merge(customers, on='customer_id', how='outer')
merged = merged[(merged['effective_from_y'] <= merged['sale_date']) & (merged['expiry_date_y'] > merged['sale_date'])]
merged_with_products = merged.merge(products, on='product_id', how='left')
data = merged_with_products.groupby(['sale_date', 'membership_status', 'name'])['total_price'].sum().unstack().fillna(0)

data

## Ticket MIDP-307

In [ ]:
import pandas as pd

# Read the sales and customer data
sales_df = pd.read_parquet('./final_data/sales.parquet')
customer_df = pd.read_parquet('./final_data/customer.parquet')

# Aggregate sales by customer
aggregated_sales = sales_df.groupby('customer_id')['quantity'].sum().reset_index()

# Merge with customer data
merged_df = pd.merge(customer_df, aggregated_sales, on='customer_id', how='inner')

# Top 10 customers by total purchased quantity
sales_report = merged_df[['first_name', 'last_name', 'email', 'quantity']].sort_values(by='quantity', ascending=False).head(10)

# Save to CSV
sales_report.to_csv('sales_report.csv', index=False)

# Filter for active members
active_member_df = merged_df[merged_df['membership_status'] == 'active']

# Top 10 active members by purchased quantity
sales_report_active_member = active_member_df[['first_name', 'last_name', 'email', 'membership_status','quantity']].sort_values(by='quantity', ascending=False).head(10)

# Save to CSV
sales_report_active_member.to_csv('sales_report_active_member.csv', index=False)

# Display results
print("Top 10 Customers by Sales Quantity:")
print(sales_report)

print("Top 10 Active Members by Sales Quantity:")
print(sales_report_active_member)
